# Jenkins와 argo용으로 리빌딩
### 그래봐야 달라지는 건 없음

### 전과 똑같이 eks의 network 형성을 위한 2번의 bootstrapping을 거침
### 거기서 kubeflow 관련된 파트만 지우고 jenkins_argo 등 파트를 새 파일로 추가할 예정

# 인스턴스 사이즈는 줄여야 됨

- eks의 구성조건, pod network, aws controller 등의 설치 과정이 node 숫자의 변화에 민감하고,   
- infra 구성 완료 전에 일어 나면 안되고  
- infra 구성 후 node 숫자를 조절 하면서 실행할 필요가 있어 이를 ad-hoc script로 남깁니다.

# 클러스터 구성 과정
# 1. node 0개로 클러스터 생성 (secondary cidr 설정을 위해)
# 2. bootstrap (secondary cidr, aws load balancer controller)
# 3. 노드 갯수 원하는 양으로 조절
## 의 과정을 거쳐야 한다. (aws는 k8s에 최적화가 덜되서...)

절차

1. node_size.yml => 0
2. terraform apply => output으로 eniconfig.yaml 파일 작성
3. boot strap 과정에 개편을 거쳐서 이젠 boot strap 데이터를 terraform 으로 전송해서, null_resource를 통해 실행시킨다.
4. node_size.yml => cluster size you want
5. terraform apply --auto-approve
6. node_size.yml => 원하는 사이즈
7. terraform apply --auto-approve

In [1]:

"""
kubeflow 를 설치후 node=0 로 만들고 다시 키니까
=> block storage가 떨어진 후 다시 안붙음
=> 어? 왜 주소가 달라 너 안붙여줄거야
=> 아니 나라니까?
=> 그럼 다시 만들기라도 해야 되는데 /안만들고 그냥 에러남
=> secondary cidr와 install kubeflow의 bootstrap을 쪼개기로 결정
"""

import os
def change_current_state_for_bootstrap(state="secondary_cidr"):
    # There is two state, making secondary cidr for pod
    # Another is bootstrap for aws controllers and kubeflow

    if state == "secondary_cidr" :
        if os.path.isfile("bootstrap_ctrl_and_kf.tf"):
            os.rename("bootstrap_ctrl_and_kf.tf", "bootstrap_ctrl_and_kf.tf.txt")
        else :
            print("file already exist")
    elif state == "bootstrap" :
        if os.path.isfile("bootstrap_ctrl_and_kf.tf.txt"):
            os.rename("bootstrap_ctrl_and_kf.tf.txt", "bootstrap_ctrl_and_kf.tf")
        else :
            print("file already exist")
    else:
        print("something wrong in bootstrap file name")

In [2]:
!terraform destroy --auto-approve

# 1. 초기 클러스터 크기

In [3]:
import yaml
def change_node_size_yml(node_size_file, desired_size, max_size, min_size):
    print("node size file", node_size_file)
    with open(node_size_file) as f:
        node_size = yaml.load(f, Loader=yaml.FullLoader)
        print('current_state',node_size)
        
    node_size['desired_size'] = desired_size
    node_size['min_size'] = min_size
    node_size['max_size'] = max_size
    
    print('after change', node_size)
    
    with open(node_size_file, 'w') as f:
        yaml.dump(node_size,f)

In [4]:
change_node_size_yml(node_size_file = 'node_size_for_group_1.yml',
                     desired_size = 0, 
                     max_size = 1, #max_size는 0이 될 수 없다. 
                     min_size = 0)

node size file node_size_for_group_1.yml
current_state {'desired_size': 0, 'max_size': 1, 'min_size': 0}
after change {'desired_size': 0, 'max_size': 1, 'min_size': 0}


In [5]:
change_node_size_yml(node_size_file = 'node_size_for_group_2.yml',
                     desired_size = 0, 
                     max_size = 1, #max_size는 0이 될 수 없다. 
                     min_size = 0)

node size file node_size_for_group_2.yml
current_state {'desired_size': 0, 'max_size': 1, 'min_size': 0}
after change {'desired_size': 0, 'max_size': 1, 'min_size': 0}


In [6]:
change_current_state_for_bootstrap(state="secondary_cidr") 

file already exist


# 2. Terraform 실행

#테라폼 설치 법 2023-06-25 현재 최신버전  1.5.1    
https://askubuntu.com/questions/983351/how-to-install-terraform-in-ubuntu

In [7]:
#mv terraform /usr/local/bin #컨테이너 안에서 실행했음

In [8]:
#!terraform --version

In [9]:
#!terraform init

In [10]:
#!terraform plan

In [3]:
!terraform apply --auto-approve

null_resource.bootstrap_secondary_cidr (remote-exec): Scanning linux images... [========     ]
null_resource.bootstrap_secondary_cidr (remote-exec): Scanning linux images... [=============]
null_resource.bootstrap_secondary_cidr (remote-exec): Scanning linux images...

null_resource.bootstrap_secondary_cidr (remote-exec): Running kernel seems to be up-to-date.

null_resource.bootstrap_secondary_cidr (remote-exec): Restarting services...
null_resource.bootstrap_secondary_cidr (remote-exec        (BPackage configuration─┤ Daemons using outdated libraries ├───
null_resource.bootstrap_secondary_cidr (remote-exec): 
null_resource.bootstrap_secondary_cidr (remote-exec): hich services should be
null_resource.bootstrap_secondary_cidr (remote-exec): estarted?
null_resource.bootstrap_secondary_cidr (remote-exec): 
null_resource.bootstrap_secondary_cidr (remote-exec):   [*] multipathd.service   
null_resource.bootstrap_secondary_cidr (remote-exec):   [*] packagekit.service   
null_resource.bootstrap_secondary_cidr (remote-exec):   [ ] unattended-upgra...  
null_resource.bootstrap_secondary_cidr (remote-exec): 
null_resource.bootstrap_secondary_cidr (remote-exec): 
null_resource.bootstrap_secondary_cidr (remote-exec):         <Ok>              <Cancel>
null_resource.bootstrap_secondary_cidr (remote-exec): 
null_resource.bootstrap_secondary_cidr (remote-exec): ────────────────────────────────────────                                                  (BPackage configuration─┤ Daemons using outdated libraries ├───
null_resource.bootstrap_secondary_c  id                   이게 뭔가 코드가 좀 바뀌었나봄. => ser  
뭔가 ok를 눌러야 되는걸로 바뀌었나봄 킹받네;;;  


sudo NEEDRESTART_MODE=a apt install -y awscli  
이런식으로 하면 해결
                   

그러네 업로드 다운로드 리스트 확인 등등에 출력을 해놔야 파이프라인에서 리스트를 확인 할 수 있구나

In [ ]:
import time

print('wait for 3 min')

time.sleep(160)

# 3. bastion server에서 bootstrap 실행
# => 테라폼 null_resource로 대체

# 4 node size -> 다시 늘리기

In [ ]:
#node_size.yml => cluster size you want

In [ ]:
#kubeflow 자체가 어짜피 scalable하게 안만들어져 있어서 관련 pod들은 한군데로 몰아놓음
change_node_size_yml(node_size_file = 'node_size_for_group_1.yml',
                     desired_size = 1, 
                     max_size = 1, #max_size는 0이 될 수 없다. 
                     min_size = 1)

In [ ]:
#이건 scale out할때만 늘리면 됨
change_node_size_yml(node_size_file = 'node_size_for_group_2.yml',
                     desired_size = 1, 
                     max_size = 1, #max_size는 0이 될 수 없다. 
                     min_size = 1)

In [ ]:
change_current_state_for_bootstrap(state="bootstrap")

In [ ]:
!terraform apply --auto-approve

In [ ]:
#잠깐 쉴때는 노드 사이즈만 줄여도 괜찮네
#!terraform apply --auto-approve

In [ ]:
여기 이전에 멈춰~~~~

In [ ]:
ingress로 바꾸면 마지막에 bastion ip랑 lb ip 띄워야 겠지

In [ ]:
#bastion sg 추가
#!terraform apply --auto-approve

In [ ]:
저번에 보니까 eks driver들을 terraform에서 설정 가능하던데...

In [ ]:
#!terraform apply --auto-approve

# 5 scale out 할때

In [ ]:
#kubeflow 자체가 어짜피 scalable하게 안만들어져 있어서 관련 pod들은 한군데로 몰아놓음
change_node_size_yml(node_size_file = 'node_size_for_group_1.yml',
                     desired_size = 1, 
                     max_size = 1, #max_size는 0이 될 수 없다. 
                     min_size = 1)

In [ ]:
#이건 scale out할때만 늘리면 됨
#next time, i have to scale out instances by 10 <- it looks too much... but it's okay!
change_node_size_yml(node_size_file = 'node_size_for_group_2.yml',
                     desired_size = 1,
                     max_size = 1, #max_size는 0이 될 수 없다. 
                     min_size = 1)

In [ ]:
!terraform apply --auto-approve

# 6 destroy

In [ ]:
!terraform destroy --auto-approve